<a href="https://colab.research.google.com/github/Praxis-bs/DEMD/blob/main/WebApps_for_ML_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://4.bp.blogspot.com/-gbL5nZDkpFQ/XScFYwoTEII/AAAAAAAAAGY/CcVb_HDLwvs2Brv5T4vSsUcz7O4r2Q79ACK4BGAYYCw/s1600/kk3-header00-beta.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Introduction

This exercise is loosely modelled on the tutorial given in https://pythonspot.com/flask-web-app-with-python/ except that instead of working with a local machine where the TCPIP ports are easily accessible, we use Google Colab where such ports are not easily accessible. To overcome this problem we use ngrok tunnelling technology that creates a publicly accessible URL that can access TCPIP applications running on any Google Colab VM port <br>
All files used in these exercises are availale at https://drive.google.com/drive/folders/1SzeS5AqnCNa9FTSeUHCGcJsClrGNlGVt?usp=sharing <br> <br>
Other ways of achieving this are as follows 


*   flask-ngrok - https://pypi.org/project/flask-ngrok/
*   pyngrok - https://pypi.org/project/pyngrok/
<br>
However the MOST SIMPLEST way to access a port in google colab is to follow the instructions given in <br>
 https://stackoverflow.com/questions/59741453/is-there-a-general-way-to-run-web-applications-on-google-colab <br>

see https://colab.research.google.com/drive/1XIz5OE3CqvlqjyYPQxrO-zhlVa8ZhMA0?usp=sharing


#ngrok configuration

ngrok needs an authentication token. this can be easily hardcoded into this notebook ( as shown in one of the cells below) To avoid hardcoding -- and revealing the same in a public notebook -- the token is stored in Google Drive and accessed by mounting the G-Drive on the Colab VM <br>
See 


1.   https://ngrok.com/ - login 
2.   https://dashboard.ngrok.com/get-started/setup - get your authtoken



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#!ls /content/drive/'My Drive'/Praxis/WebCredentials
!cp /content/drive/'My Drive'/Praxis/WebCredentials/ngrokToken.py .

In [3]:
# credential file for Prithwis Mukerjee
# this file needs to be uploaded into the VM

from ngrokToken import ngrokToken

#for the sake of privacy
#the following credentials need to be stored in a text file called ngrokToken.py
#in the format given below
#in the Colab VM

#otherwise, the values can be directly placed here

#ngrokToken = '1kUOcwwwwwwwwRHsdVJRGMeTdddddddddddYbPZhxod'   # uncomment this line and place your own credentials here
#token is available at https://dashboard.ngrok.com/get-started/setup


#print(ngrokToken)
ngrokTokenCmd = 'ngrok authtoken '+ngrokToken

In [4]:
#https://ngrok.com/download
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2021-08-24 15:30:32--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.45.123.219, 44.194.107.75, 44.193.92.248, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.45.123.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  26.9MB/s    in 0.5s    

2021-08-24 15:30:32 (26.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]



In [5]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [6]:
#!./ngrok authtoken xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
get_ipython().system_raw(ngrokTokenCmd)

In [7]:
!apt-get install jq > null

#WebApp Linear Regression - Wine Analysis

Web apps need additionl files for HTML, images, data etc that need to be stored on appropriate directories. These files can either be created in a local machine and uploaded into the appropriate directory, or loaded from GDrive or Github

In [10]:
!mkdir static
!mkdir templates
!mkdir datafiles

mkdir: cannot create directory ‘static’: File exists


In [ ]:
!wget https://github.com/Praxis-bs/DEMD/raw/main/static/kkpraxis.png      # image file

!wget https://raw.githubusercontent.com/Praxis-bs/DEMD/main/templates/LR_home.html    # home
!wget https://raw.githubusercontent.com/Praxis-bs/DEMD/main/templates/LR_predict1.html  # output

!wget https://github.com/Praxis-bs/DEMD/raw/main/datafiles/lrw-model.pkl # model file

In [11]:
!mv kkpraxis.png static
!mv *.html templates/
!mv *.pkl datafiles/

--2021-08-24 15:34:36--  https://github.com/Praxis-bs/DEMD/raw/main/static/kkpraxis.png
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Praxis-bs/DEMD/main/static/kkpraxis.png [following]
--2021-08-24 15:34:36--  https://raw.githubusercontent.com/Praxis-bs/DEMD/main/static/kkpraxis.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33605 (33K) [image/png]
Saving to: ‘kkpraxis.png’

kkpraxis.png        100%[===================>]  32.82K  --.-KB/s    in 0.002s  

2021-08-24 15:34:37 (15.6 MB/s) - ‘kkpraxis.png’ saved [33605/33605]



In [17]:
get_ipython().system_raw('./ngrok http 5010 &')

In [18]:
!curl -s http://localhost:4040/api/tunnels | jq .tunnels[0].public_url
!curl -s http://localhost:4040/api/tunnels | jq .tunnels[0].config.addr

"https://437f-34-73-118-25.ngrok.io"
"http://localhost:5010"


In [19]:
# https://pythonspot.com/flask-web-app-with-python/
# COLAB where model is trained, tested
# Train : https://colab.research.google.com/drive/1G3VUKVUspcoDnLRobuUfsGTswpCMkU8W
# Predict : https://colab.research.google.com/drive/1gF9DI4NT5tH_9lcbVth-kVEZ9sO7wnhU
# docker build -t calcutta/de-applrw02 .
# docker run --rm --name praxislrw -p 5002:5000 calcutta/de-applrw02

from sys import argv
from flask import Flask, flash, redirect, render_template, request, session, abort

import pickle
from sklearn import linear_model

app = Flask(__name__)
scriptName = argv[0]

@app.route("/")
def hello():
    retString = 'ML LRW now running ... '+scriptName
    return retString

@app.route('/home')
def home():
    return render_template('LR_home.html')



@app.route('/predict1/<featureString>')
def multidata(featureString):
    str1 = featureString
    feature_list = featureString.split(",")
    feature_array = list(map(lambda x : float(x), feature_list))

    model = pickle.load(open("datafiles/lrw-model.pkl","rb"))

    #our model rates the wine based on the input array
    prediction = model.predict([feature_array]).tolist()

    return render_template('LR_predict1.html',**locals())


if __name__ == "__main__":
    app.run(host="localhost", port=5010)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5010/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Aug/2021 15:52:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2021 15:52:09] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [24/Aug/2021 15:52:15] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2021 15:52:15] "GET /static/kkpraxis.png HTTP/1.1" 200 -
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 0.18.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  U

#ML - Image Classification <br>
https://blog.hyperiondev.com/index.php/2018/02/01/deploy-machine-learning-model-flask-api/

In [ ]:
#!cp /content/drive/'My Drive'/Praxis/'Course - DevOps'/DevOps/ml/appimclass02/datafiles/*.pkl datafiles
#https://drive.google.com/file/d/1o6Z02Q-KATgEXSGbQ8XK_H62Dv7BW4eW/view?usp=sharing
!gdown https://drive.google.com/uc?id=1o6Z02Q-KATgEXSGbQ8XK_H62Dv7BW4eW

#!cp /content/drive/'My Drive'/Praxis/'Course - DevOps'/DevOps/ml/appimclass02/templates/index.html templates 
#https://drive.google.com/file/d/1pNiq27_abII3mj8PFui1dn1mrKCLlPKd/view?usp=sharing
!gdown https://drive.google.com/uc?id=1pNiq27_abII3mj8PFui1dn1mrKCLlPKd

In [ ]:
!mv *.pkl datafiles/
!mv index.html templates/

In [ ]:
get_ipython().system_raw('./ngrok http 5010 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | jq .tunnels[0].public_url
!curl -s http://localhost:4040/api/tunnels | jq .tunnels[0].config.addr

In [ ]:
# https://pythonspot.com/flask-web-app-with-python/
# https://blog.hyperiondev.com/index.php/2018/02/01/deploy-machine-learning-model-flask-api/
# COLAB for train, test, model build https://colab.research.google.com/drive/1JuTezUiqM0Cj_m-FmmeBzw_88xZlYeK-
# docker build -t calcutta/de-appimc02 .
# docker run --rm --name praxisimc -p 5000:5000 calcutta/de-appimc02

# test images available at https://github.com/elliebirbeck/model-deployment-flask/tree/master/test-imgs


from sys import argv
from flask import Flask, flash, redirect, render_template, request, session, abort, url_for

import pickle
from sklearn import linear_model
#from sklearn.externals import joblib
import joblib
from sklearn.ensemble import *

import numpy as np
import imageio


app = Flask(__name__)
scriptName = argv[0]

@app.route("/")
def hello():
    retString = 'now running ... '+scriptName
    return retString

@app.route('/home')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def make_prediction():
    if request.method=='POST':
        file = request.files['image']
        if not file: 
            return render_template('index.html', label="No file")
        #f = 'datafiles/'+str(file.filename)
        #file.save(f)
        
        img = imageio.imread(file)
        img = img[:,:,:3]
        img = img.reshape(1, -1)
        prediction = model.predict(img)
        label = str(np.squeeze(prediction))
        if label=='10': 
            label='0'
        return render_template('index.html', label=label, file=file)

if __name__ == "__main__":
    model = joblib.load('datafiles/imClass-model-18032020.pkl')
    app.run(host="localhost", port=5010)

#ML Tensorflow + Keras deprecated <br>
too many changes in versions etc .. <br>
this app does not work anymore 

In [ ]:
!cp /content/drive/'My Drive'/Praxis/'Course - DevOps'/DevOps/ml/apptfk2/games-17032020.h5 datafiles
!cp /content/drive/'My Drive'/Praxis/'Course - DevOps'/DevOps/ml/apptfk2/templates/home.html templates/home_tfk.html
!cp /content/drive/'My Drive'/Praxis/'Course - DevOps'/DevOps/ml/apptfk2/templates/predict2.html templates

In [ ]:
get_ipython().system_raw('./ngrok http 5010 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | jq .tunnels[0].public_url
!curl -s http://localhost:4040/api/tunnels | jq .tunnels[0].config.addr

In [ ]:
# tutorial from https://www.houseofbots.com/news-detail/4528-1-how-to-deploy-keras-deep-learning-models-with-flask
# model is built in colab https://colab.research.google.com/drive/1sn7BS1uqPdmiAb2Ez1Q0EcpwwNeioCY0
# docker build -t calcutta/de-apptfk02 .
# docker run --rm --name praxis007 -p 8888:5000 calcutta/de-apptfk02
# jsonify error see https://stackoverflow.com/questions/60131900/weird-is-xhr-error-when-deploying-flask-app-to-heroku

from sys import argv

# Load libraries

import flask
from flask import Flask, flash, redirect, render_template, request, session, abort
import pandas as pd
#import tensorflow as tf
#import keras
#from keras.models import load_model


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model

# instantiate flask 
app = flask.Flask(__name__)
#app = Flask(__name__)
scriptName = argv[0]

# we need to redefine our metric function in order 
# to use it when loading the model 
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    keras.backend.get_session().run(tf.local_variables_initializer())
    return auc
# load the model, and pass in the custom metric function
global graph
#graph = tf.get_default_graph()
graph = tf.compat.v1.get_default_graph()
#model = load_model('./games.h5', custom_objects={'auc': auc})
model = load_model('datafiles/games-17032020.h5', custom_objects={'auc': auc})
model.run_eagerly = True


@app.route("/")
def hello():
    retString = 'APPTFK2 running '+scriptName+' go to /home for more information'
    return retString

@app.route('/home')
def home():
    return render_template('home_tfk.html')

# define a predict function as an endpoint 
@app.route("/predict", methods=["GET","POST"])
def predict():
    data = {"success": False}
    params = flask.request.json
    
    if (params == None):
       params = flask.request.args
       
    #params = {'g1':1,'g2':0,'g3':0,'g4':0,'g5':0,'g6':0,'g7':0,'g8':0,'g9':0,'g10':0}
    # if parameters are found, return a prediction
    if (params != None):
        x=pd.DataFrame.from_dict(params, orient='index').transpose()
        with graph.as_default():
            data["prediction"] = str(model.predict(x)[0][0])
            data["success"] = True
    # return a response in json format 
    return flask.jsonify(data)  

# define a predict function as an endpoint 
@app.route("/predict2", methods=["GET","POST"])
def predict2():
    data = {"success": False}
    params = flask.request.json
    
    if (params == None):
       params = flask.request.args
       
    #params = {'g1':1,'g2':0,'g3':0,'g4':0,'g5':0,'g6':0,'g7':0,'g8':0,'g9':0,'g10':0}
    # if parameters are found, return a prediction
    if (params != None):
        x=pd.DataFrame.from_dict(params, orient='index').transpose()
        with graph.as_default():
            prediction = str(model.predict(x)[0][0])
            status = True
    return render_template('predict2.html',**locals())
      
# start the flask app, allow remote connections 
app.run(host="localhost", port=5010)

#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)